In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import scanpy
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader, Batch
from torch.utils.data import TensorDataset
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.utils import train_test_split_edges
from torch_geometric.nn import GATConv, GCNConv, SAGEConv, MessagePassing, Linear
import random
from scipy.io import mmread
import numpy as np
from sklearn.model_selection import train_test_split
from torcheval.metrics.functional import multiclass_f1_score
from sklearn.metrics import confusion_matrix
from collections import Counter

In [2]:
hicmatrix  =pd.read_csv("../../chrX_1000000_hic.txt",header = None)
# hicmatrix_test = pd.read_csv("../../chrX_2000000_hic.txt",header = None)
epigenomic = pd.read_csv("../../train_epigemnomic_1000000.csv",header =  0).iloc[:,1:].to_numpy()
label = pd.read_csv("../../train_label.csv",header = 0).iloc[:,1:].to_numpy()


In [91]:
hicmatrix_train_np

array([[  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ..., 157., 269.,  24.],
       [  0.,   0.,   0., ..., 269., 130., 233.],
       [  0.,   0.,   0., ...,  24., 233., 134.]], dtype=float32)

In [42]:
# hicmatrix_train  =pd.read_csv("../../chrX_1000000_hic.txt",header = None)
# hicmatrix_test = pd.read_csv("../../chrX_2000000_hic.txt",header = None)
# epigenomic_test = pd.read_csv("../../test_epigemnomic_2000000.csv",header =  0).iloc[:,1:].to_numpy()
# label_test = pd.read_csv("../../test_label.csv",header = 0).iloc[:,1:].to_numpy()

In [4]:
epigenomic.shape

(2001, 24)

In [5]:
print(np.unique(label, return_counts=True)
)

(array([0, 1]), array([1950,   51]))


In [6]:
hicmatrix_np =hicmatrix.to_numpy(dtype=np.float32)
# hicmatrix_test_np = hicmatrix_test.to_numpy(dtype= np.float32)

In [7]:
# label = random_array

NameError: name 'random_array' is not defined

In [26]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)
data

Data(x=[3, 1], edge_index=[2, 4])

In [8]:
# From Whitney

def create_comp_graph(correlation, min_corr):
    '''Get a graph based on highly correlated features (feat) and given correlation (corr).'''
    s_l = [] # Gene 1 = Node 1
    d_l  = [] # Gene 2 = Node 2
    for i in range(0, correlation.shape[0]):
        for j in range(0, correlation.shape[1]):# Get edge and correlation
            if correlation[i][j] >= min_corr and i != j: # Ensure the features are at least given correlation (and it won't be a self correlated edge i. e. 1 --> 1)
                s_l.append(i)
                d_l.append(j)
            else:
                continue
    print(f"Number of edges: {len(s_l)}") # Prints # of edges so you can make sure it isn't empty/too large
    edge_index = np.array([s_l, d_l]) # Convert to array
    return edge_index # Return edge index

edge_index = create_comp_graph(hicmatrix_np, 10)
temp_edge_index = edge_index
# edge_index_test = create_comp_graph(hicmatrix_test_np,10)
# temp_edge_index_test = edge_index_test

Number of edges: 61946


In [24]:
edge_index

tensor([[  21,   22,   22,  ..., 2000, 2000, 2000],
        [  22,   21,   23,  ..., 1997, 1998, 1999]])

In [9]:
edge_index

array([[  21,   22,   22, ..., 2000, 2000, 2000],
       [  22,   21,   23, ..., 1997, 1998, 1999]])

In [10]:
def set_seed(seed: int = 12) -> None:
    '''This function allows us to set the seed for the notebook.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

In [11]:
import math
# Splitting into train and test sets

# note that the training and testing matrices are the same length
# indices = np.arange(len(epigenomic_train.index))
# np.random.shuffle(indices)
# training_exp_shuffled = training_head_expression.iloc[indices].to_numpy(dtype=np.float32)
# training_age_labels = encoded_training_age.iloc[indices].to_numpy(dtype=np.int64)
# np.random.shuffle(indices)
# testing_exp_shuffled = testing_head_expression.iloc[indices].to_numpy(dtype=np.float32)
# testing_age_labels = encoded_testing_age.iloc[indices].to_numpy(dtype=np.int64)

In [12]:
epsilon = 1
# training_exp_shuffled = training_exp_shuffled + epsilon
# training_exp_shuffled = np.log2(training_exp_shuffled)
# testing_exp_shuffled = testing_exp_shuffled + epsilon
# testing_exp_shuffled = np.log2(testing_exp_shuffled)

In [14]:
# edge_index_train = torch.tensor(edge_index_train, dtype=torch.int64)
# edge_index_test = torch.tensor(edge_index_test,dtype = torch.int64)
# train_data = []
# test_data = []
edge_index = torch.tensor(edge_index, dtype = torch.int64)
data = []
# Function to convert graph data to PyTorch Geometric Data object
def create_data_object(x, y, edge_index):
    x = torch.tensor(x, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.long)
    return Data(x=x, y=y, edge_index=edge_index)

# Create training dataset
for j in range(2000):
    x = epigenomic[j]
    y = (label[j])
    #y_train = training_age_labels[j]
    data.append(create_data_object(x, y, edge_index))

# # Create training dataset
# for j in range(2000):
#     x_train = epigenomic_train[j]
#     y_train = n(label_train[j])
#     #y_train = training_age_labels[j]
#     train_data.append(create_data_object(x_train, y_train, edge_index_train))

# # Create testing dataset
# for k in range(2000):
#     x_test = epigenomic_test[k]
#     y_test = (label_test[k])
#     #y_test = testing_age_labels[k]
#     test_data.append(create_data_object(x_test, y_test, edge_index_test))

# Convert lists of Data objects to Batch objects
# train_data = Batch.from_data_list(train_data)
# test_data = Batch.from_data_list(test_data)
# data = create_data_object(training_exp_shuffled[2001], np.argmax(training_sex_labels[2001]), edge_index)
# g = to_networkx(data, to_undirected=True)
# nx.draw(g)



# # Create DataLoader for training and testing
# train_loader = DataLoader(train_data, batch_size=75, shuffle=True)
# test_loader = DataLoader(test_data, batch_size=75, shuffle=False)

# data_loader = DataLoader(train_data, batch_size=75, shuffle=True)

/tmp/ipykernel_81557/2785758134.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index = torch.tensor(edge_index, dtype = torch.int64)


In [16]:
data = DataLoader(data, batch_size = 75)

/oscar/data/larschan/gcn_gps/lib/python3.8/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


AttributeError: 'DataLoader' object has no attribute 'num_edges'

In [19]:
from torch_geometric.nn import global_mean_pool, global_max_pool
from torch_geometric.graphgym import init_weights

class GCN(torch.nn.Module):
    def __init__(self, hidden_dim1, num_labels):
        super(GCN, self).__init__()
        self.conv1 = SAGEConv(1, hidden_dim1)
        self.init_weights(self.conv1)
        # self.conv2 = SAGEConv(hidden_dim1, hidden_dim2)
        # self.init_weights(self.conv2)
        self.conv3 = SAGEConv(hidden_dim1, num_labels)
        self.init_weights(self.conv3)
        # self.conv4 = SAGEConv(hidden_dim3, num_labels)
        # self.init_weights(self.conv4)
        self.lin = Linear(num_labels, num_labels)
    def forward(self, x, edge_index, batch=None):
        print(edge_index)
        x = F.leaky_relu(self.conv1(x, edge_index))
       # x = F.leaky_relu(self.conv2(x, edge_index))
        x = F.leaky_relu(self.conv3(x, edge_index))
        # x = F.leaky_relu(self.conv4(x, edge_index))
        x = global_mean_pool(x, batch) 
        x = self.lin(x)
        x = F.log_softmax(x, dim=edge_index)
        return x

    def init_weights(self, layer):
        if hasattr(layer, 'weight') and layer.weight is not None:
            xavier_uniform_(layer.weight.data)


/oscar/data/larschan/gcn_gps/lib/python3.8/site-packages/torch_geometric/graphgym/config.py:19: UserWarning: Could not define global config object. Please install 'yacs' via 'pip install yacs' in order to use GraphGym
  warnings.warn("Could not define global config object. Please install "
/oscar/data/larschan/gcn_gps/lib/python3.8/site-packages/torch_geometric/graphgym/imports.py:14: UserWarning: Please install 'pytorch_lightning' via  'pip install pytorch_lightning' in order to use GraphGym
  warnings.warn("Please install 'pytorch_lightning' via  "


In [20]:
from sklearn.metrics import roc_auc_score
import tqdm


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_labels=4

model = GCN(64, num_labels).to(device)
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion =  nn.CrossEntropyLoss() # change loss for binary classification vs multiclass

def accuracy(predictions, targets):
    # Assuming predictions and targets are tensors
    _, predicted_classes = predictions.max(dim=1)
    correct = (predicted_classes == targets).sum().item()
    accuracy = correct / targets.size(0)
    return accuracy

def train(train_loader):
    model.train()
    avgLoss = 0
    avgAcc = 0
    for data in tqdm.tqdm(train_loader):
        optimizer.zero_grad()
        data.x = torch.reshape(data.x, (data.x.shape[0], 1)).to(torch.float32).to(device)
        # data.x = data.x.view(-1, 1).to(torch.float32)
        out = model(data.x, data.edge_index, data.batch)
        #target = torch.reshape(data.y, (100,2)) # CHANGE DIMENSIONS AS YOU CHANGE BATCH SIZE
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        avgLoss += loss
        avgAcc += accuracy(out, data.y)
    return (avgLoss / len(train_loader)), (avgAcc / len(train_loader))
def test(loader):
    model.eval()
    correct = 0
    for data in loader: 
            # data.x = data.x.view(-1, 1).to(torch.float32)
        data.x = torch.reshape(data.x, (data.x.shape[0], 1)).to(torch.float32).to(device)
        out = model(data.x, data.edge_index, data.batch)
        #target = torch.reshape(data.y, (100,2))
        pred = torch.argmax(out, dim=1)
        correct += int((pred == data.y).sum())
    return correct / len(loader.dataset)

GCN(
  (conv1): SAGEConv(1, 64, aggr=mean)
  (conv3): SAGEConv(64, 4, aggr=mean)
  (lin): Linear(4, 4, bias=True)
)


In [23]:
for epoch in range(1,21):
    loss, train_acc = train(data)
#     test_acc = test(test_loader)
    print(f'Epoch: {epoch}, Train: {train_acc}, Test Accuracy: {test_acc}')

  0%|          | 0/27 [00:00<?, ?it/s]


tensor([[  21,   22,   22,  ..., 3776, 3776, 3776],
        [  22,   21,   23,  ..., 3773, 3774, 3775]])


IndexError: Encountered an index error. Please ensure that all indices in 'edge_index' point to valid indices in the interval [0, 1799] (got interval [21, 3776])

In [107]:
train_loader